In [73]:
import sys
import os
sys.path.insert(0, os.path.abspath('/home/maldo/projects/virtualenvs/datathon/lib/python3.7/site-packages'))
sys.path.insert(0, os.path.abspath('/home/maldo/projects/datathon'))
from imblearn.under_sampling import *
from imblearn.over_sampling import *
from imblearn.combine import *
from src.models.model import SoilClassifier
from src.constants import *
from sklearn.base import clone
from src.features.features import ImbalanceTransformer
import pandas as pd
import seaborn as sns
import math

In [74]:
df = pd.read_csv(os.path.join('../data/processed', 'train_data.csv'))
df_test = pd.read_csv(os.path.join('../data/processed', 'test_data.csv'))
min_samples = 500
max_samples = 9000
features = ['not_correlated', 'cadastral_ordinal_encoder_onehot', 'log_area', 'log_antiquity', 'squared_geoms', 'pssr', 'savi']
classifier = 'gradient_boosting'

In [77]:
model = SoilClassifier(min_samples = min_samples,
                       max_samples = max_samples,
                       feature_names = features,
                       classifier = classifier)

In [78]:
model.fit(df, df[TARGET_FEATURE])

In [11]:
pipe = clone(model.pipeline)
clsf = pipe.steps.pop()
imb = pipe.steps.pop()

In [12]:
transformed_data = pipe.fit_transform(df)

In [13]:
transformed_data = pd.DataFrame(transformed_data, columns=pipe.named_steps['features'].get_feature_names())

In [14]:
imbt = ImbalanceTransformer(min_samples=min_samples, max_samples=max_samples)

In [ ]:
X_train ,y_train = imbt.fit_resample(transformed_data, df[TARGET_FEATURE])

In [69]:
X_test = pipe.transform(df_test)
X_test = pd.DataFrame(X_test, columns=pipe.named_steps['features'].get_feature_names())
y_test = df_test[TARGET_FEATURE]

In [17]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(19597, 40) (19597,)
(20646, 40) (20646,)


In [71]:
X_train.to_csv('../data/interim/X_train.csv', index=False)
y_train.to_csv('../data/interim/y_train.csv', index=False)
X_test.to_csv('../data/interim/X_test.csv', index=False)
y_test.to_csv('../data/interim/y_test.csv', index=False)

# Results Comparision

In [5]:
pd.read_csv('../src/experiments/results/classifierComparisons/results.csv').sort_values('test_custom_accuracy', ascending=False)

,best_params,model,test_accuracy,test_balanced_accuracy,test_custom_accuracy,test_f1_macro,test_f1_weighted,test_precision_macro,test_precision_weighted,test_recall_macro,test_recall_weighted,train_accuracy,train_balanced_accuracy,train_custom_accuracy,train_f1_macro,train_f1_weighted,train_precision_macro,train_precision_weighted,train_recall_macro,train_recall_weighted
0,"{'max_depth': 7, 'max_features': None, 'min_sa...",gradient_boosting,0.8614743776034098,0.503262,0.669340,0.480835,0.869747,0.474157,0.881656,0.503262,0.861474,0.829464,0.758908,0.742639,0.800947,0.821070,0.874306,0.837151,0.758908,0.829464
3,"{'max_depth': 7, 'max_features': None, 'min_sa...",random_forest,0.8492201879298653,0.412060,0.624689,0.399960,0.853495,0.515409,0.877104,0.412060,0.849220,0.658009,0.478290,0.485627,0.504643,0.613682,0.684980,0.666146,0.478290,0.658009
4,"{'kernel': 'linear', 'gamma': 0.001, 'C': 100}",support_vector_machine,0.8474765087668313,0.381993,0.586559,0.346485,0.843154,0.338716,0.842352,0.381993,0.847477,0.603000,0.411168,0.397543,0.415546,0.541844,0.572980,0.588513,0.411168,0.603000
1,"{'solver': 'svd', 'alpha': 1}",ridge,0.863508669ju9602829,0.288129,0.558973,0.298245,0.844236,0.424419,0.835680,0.288129,0.863509,0.573455,0.299191,0.315293,0.307139,0.488095,0.490419,0.507149,0.299191,0.573455
2,"{'shrinkage': 0.09845651447194592, 'solver': '...",lda,0.8302818947980238,0.330487,0.551515,0.281280,0.827109,0.288782,0.828526,0.330487,0.830282,0.566719,0.362188,0.345080,0.357705,0.497773,0.419672,0.489156,0.362188,0.566719
